In [ ]:
# Executar esta célula primeiro
import json
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

ARQUIVO_TAREFAS = "tarefas_obra.json"

def carregar_tarefas():
    if not os.path.exists(ARQUIVO_TAREFAS):
        return []
    with open(ARQUIVO_TAREFAS, "r", encoding="utf-8") as f:
        return json.load(f)

def salvar_tarefas(tarefas):
    with open(ARQUIVO_TAREFAS, "w", encoding="utf-8") as f:
        json.dump(tarefas, f, indent=4, ensure_ascii=False)


In [ ]:
def adicionar_tarefa(descricao, responsavel, prioridade):
    data = datetime.now().strftime("%d/%m/%Y")
    tarefa = {
        "data": data,
        "descricao": descricao,
        "responsavel": responsavel,
        "prioridade": prioridade.lower(),
        "status": "pendente"
    }
    tarefas = carregar_tarefas()
    tarefas.append(tarefa)
    salvar_tarefas(tarefas)

def listar_tarefas():
    tarefas = carregar_tarefas()
    if not tarefas:
        print("Nenhuma tarefa cadastrada.")
        return
    for i, t in enumerate(tarefas, start=1):
        print(f"[{i}] {t['descricao']} ({t['status']})")
        print(f"   Data: {t['data']} | Resp: {t['responsavel']} | Prioridade: {t['prioridade']}\n")

def atualizar_status(indice, novo_status):
    tarefas = carregar_tarefas()
    if 0 <= indice < len(tarefas):
        tarefas[indice]['status'] = novo_status
        salvar_tarefas(tarefas)

def remover_tarefa(indice):
    tarefas = carregar_tarefas()
    if 0 <= indice < len(tarefas):
        tarefa = tarefas.pop(indice)
        salvar_tarefas(tarefas)
        print(f"Tarefa '{tarefa['descricao']}' removida!")


In [ ]:
# Widgets de entrada
descricao = widgets.Text(description="Descrição:")
responsavel = widgets.Text(description="Responsável:")
prioridade = widgets.Dropdown(options=["baixa", "média", "alta"], description="Prioridade:")
botao_add = widgets.Button(description="Adicionar Tarefa", button_style="success")
botao_listar = widgets.Button(description="Listar Tarefas", button_style="info")
botao_limpar = widgets.Button(description="Limpar Saída", button_style="warning")

saida = widgets.Output()

# Ações dos botões
def ao_clicar_adicionar(b):
    with saida:
        clear_output()
        adicionar_tarefa(descricao.value, responsavel.value, prioridade.value)
        print("✅ Tarefa adicionada com sucesso!")
        descricao.value = ""
        responsavel.value = ""

def ao_clicar_listar(b):
    with saida:
        clear_output()
        listar_tarefas()

def ao_clicar_limpar(b):
    with saida:
        clear_output()

botao_add.on_click(ao_clicar_adicionar)
botao_listar.on_click(ao_clicar_listar)
botao_limpar.on_click(ao_clicar_limpar)

# Mostrar a interface
display(widgets.VBox([
    widgets.HTML("<h3>🧱 Gerenciador de Tarefas da Obra</h3>"),
    descricao, responsavel, prioridade,
    widgets.HBox([botao_add, botao_listar, botao_limpar]),
    saida
]))


In [ ]:
indice = widgets.IntText(description="Nº tarefa:")
novo_status = widgets.Dropdown(
    options=["pendente", "em andamento", "concluída"],
    description="Novo status:"
)
botao_status = widgets.Button(description="Atualizar Status", button_style="primary")
botao_remover = widgets.Button(description="Remover Tarefa", button_style="danger")

saida2 = widgets.Output()

def ao_clicar_status(b):
    with saida2:
        clear_output()
        tarefas = carregar_tarefas()
        if 0 < indice.value <= len(tarefas):
            atualizar_status(indice.value - 1, novo_status.value)
            print("🔄 Status atualizado com sucesso!\n")
            listar_tarefas()  # ✅ Mostra a lista atualizada
        else:
            print("⚠️ Número de tarefa inválido!")

def ao_clicar_remover(b):
    with saida2:
        clear_output()
        tarefas = carregar_tarefas()
        if 0 < indice.value <= len(tarefas):
            remover_tarefa(indice.value - 1)
            print("🗑️ Tarefa removida!\n")
            listar_tarefas()  # ✅ Mostra a lista atualizada
        else:
            print("⚠️ Número de tarefa inválido!")

botao_status.on_click(ao_clicar_status)
botao_remover.on_click(ao_clicar_remover)

display(widgets.VBox([
    widgets.HTML("<h4>⚙️ Atualizar ou Remover</h4>"),
    widgets.HBox([indice, novo_status]),
    widgets.HBox([botao_status, botao_remover]),
    saida2
]))
